In [1]:
import pandas as pd
import numpy as ny
import glob
import pyarrow.parquet as pq
import datetime
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

In [2]:
#Lê dataframe criado com histórico de emprestimos e filtra apenas registros de alunos de pós graduação para analises posteriores
emprestimos = pd.read_csv('/Users/Clarice Nascimento/0. Alura/emprestimos.csv')
emprestimos_pg = emprestimos.query('tipo_vinculo_usuario == "ALUNO DE PÓS-GRADUAÇÃO"')

In [3]:
#Importando base de dados em excel com as informações dos alunos contendo numero de matricula e curso por ano, agregando em unico dataframe
matricula_alunos_2010 = pd.read_excel('/Users/Clarice Nascimento/0. Alura/d6/matricula_alunos.xlsx',sheet_name='Até 2010',skiprows=1)
matricula_alunos_2011 = pd.read_excel('/Users/Clarice Nascimento/0. Alura/d6/matricula_alunos.xlsx',sheet_name='Após 2010',skiprows=1)
matricula_alunos_2010 = matricula_alunos_2010.rename(columns={'Número da matrícula': 'Matrícula','Tipo de vinculo':'Vinculo','Curso':'Curso'})
matricula_alunos_2011 = matricula_alunos_2011.rename(columns={'Matricula ': 'Matrícula','Tipo de vinculo':'Vinculo','Cursos':'Curso'})
matricula_alunos = pd.concat([matricula_alunos_2010,matricula_alunos_2011], ignore_index=True)

In [4]:
#Importando base de dados em json com as informações dos alunos de pós graduação contendo numero de matricula
cadastro_alunos = pd.read_json('/Users/Clarice Nascimento/0. Alura/d6/cadastro_alunos.json')
cadastro_alunos = pd.read_json(cadastro_alunos.registros[1])
cadastro_alunos = cadastro_alunos.rename(columns={'matricula_ou_siape': 'Matrícula','tipo_vinculo_usuario':'Vinculo','curso':'Curso'})
cadastro_alunos

C:\Users\Clarice Nascimento\AppData\Local\Temp\ipykernel_77804\3383879447.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  cadastro_alunos = pd.read_json(cadastro_alunos.registros[1])


,Matrícula,Vinculo,Curso
0,2008112114,ALUNO DE PÓS-GRADUAÇÃO,CIÊNCIAS ODONTOLÓGICAS
1,2008106583,ALUNO DE PÓS-GRADUAÇÃO,ENGENHARIA DE SOFTWARE
2,2011101417,ALUNO DE PÓS-GRADUAÇÃO,ARQUITETURA E URBANISMO
3,2011113122,ALUNO DE PÓS-GRADUAÇÃO,FILOSOFIA
4,2010110903,ALUNO DE PÓS-GRADUAÇÃO,ARTES CÊNICAS
...,...,...,...
19435,20201013501,ALUNO DE PÓS-GRADUAÇÃO,GESTÃO DA INFORMAÇÃO E DO CONHECIMENTO
19436,20201014062,ALUNO DE PÓS-GRADUAÇÃO,"CIÊNCIA, TECNOLOGIA E INOVAÇÃO"
19437,20201017690,ALUNO DE PÓS-GRADUAÇÃO,CIÊNCIAS DA SAÚDE
19438,20201014465,ALUNO DE PÓS-GRADUAÇÃO,DESIGN


In [5]:
#Concatenando os dataframes obtidos com a importação de arquivos em excel e json para um unico dataframe
info_alunos = pd.concat([matricula_alunos,cadastro_alunos], ignore_index=True)
info_alunos

,Matrícula,Vinculo,Curso
0,1.900017e+08,ALUNO DE GRADUAÇÃO,MATEMÁTICA
1,1.990145e+08,ALUNO DE GRADUAÇÃO,CIÊNCIAS BIOLÓGICAS
2,1.997357e+08,ALUNO DE GRADUAÇÃO,MATEMÁTICA
3,1.998124e+08,ALUNO DE GRADUAÇÃO,LETRAS
4,1.998237e+08,ALUNO DE GRADUAÇÃO,ECOLOGIA
...,...,...,...
29435,2.020101e+10,ALUNO DE PÓS-GRADUAÇÃO,GESTÃO DA INFORMAÇÃO E DO CONHECIMENTO
29436,2.020101e+10,ALUNO DE PÓS-GRADUAÇÃO,"CIÊNCIA, TECNOLOGIA E INOVAÇÃO"
29437,2.020102e+10,ALUNO DE PÓS-GRADUAÇÃO,CIÊNCIAS DA SAÚDE
29438,2.020101e+10,ALUNO DE PÓS-GRADUAÇÃO,DESIGN


In [6]:
#Incluindo o curso dos alunos no dataframe de observação
alunos_pg = emprestimos_pg.merge(info_alunos,  how='left', left_on='matricula_ou_siape', right_on='Matrícula')
alunos_pg = alunos_pg.drop(columns=['Matrícula','Vinculo'])
#Filtrando somente registros a partir do ano de 2017 e cursos selecionados para analise
alunos_pg['data_emprestimo'] = pd.to_datetime(alunos_pg['data_emprestimo']) 
alunos_pg = alunos_pg.query('data_emprestimo > 2017')
alunos_pg

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema,descricao,Curso
11,9610280,L167050,2017/04/10 10:38:20.522000000,2017-03-28 09:04:09.781,2017/04/24 20:01:32.000000000,2.017103e+10,ALUNO DE PÓS-GRADUAÇÃO,70865,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647,25009,Ciências aplicadas,FILOSOFIA
21,2616635,2007027500,NaN,2018-03-16 10:01:49.789,2018/04/09 15:29:49.000000000,2.018101e+10,ALUNO DE PÓS-GRADUAÇÃO,225646,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,686,91736,Ciências aplicadas,BIOINFORMÁTICA
42,2599800,L202783,NaN,2018-02-26 14:42:38.950,2018/03/06 17:52:19.000000000,2.016103e+10,ALUNO DE PÓS-GRADUAÇÃO,274136,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,643,120537,Ciências aplicadas,BIOQUÍMICA
43,2621100,2007035128,2018/04/19 00:33:06.743000000,2018-03-21 13:38:16.932,2018/05/16 17:54:24.000000000,2.018102e+10,ALUNO DE PÓS-GRADUAÇÃO,235887,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630,97666,Ciências aplicadas,GESTÃO DA INFORMAÇÃO E DO CONHECIMENTO
44,2668516,2007035128,NaN,2018-05-25 19:01:42.414,2018/07/12 11:11:26.000000000,2.017100e+10,ALUNO DE PÓS-GRADUAÇÃO,235887,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630,97666,Ciências aplicadas,GESTÃO DA INFORMAÇÃO E DO CONHECIMENTO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302536,2988811,2015033958,2020/04/26 11:43:40.087000000,2020-03-17 16:20:01.906,2021/06/09 12:33:33.000000000,2.016103e+10,ALUNO DE PÓS-GRADUAÇÃO,1211606,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,541,227626,Matemática e ciências naturais,CIÊNCIAS ODONTOLÓGICAS
302537,2988812,2016019436,2020/04/26 11:43:40.062000000,2020-03-17 16:20:01.992,2021/06/09 12:33:39.000000000,2.016103e+10,ALUNO DE PÓS-GRADUAÇÃO,1235460,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,511,233523,Matemática e ciências naturais,CIÊNCIAS ODONTOLÓGICAS
302538,2988851,2014061383,NaN,2020-03-17 16:46:55.065,2021/10/18 11:33:35.000000000,2.018102e+10,ALUNO DE PÓS-GRADUAÇÃO,1164548,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,657,153156,Ciências aplicadas,CIÊNCIA E ENGENHARIA DE MATERIAIS
302539,2988891,2017008971,2020/04/19 16:42:15.125000000,2020-03-17 17:13:00.546,NaN,2.018102e+10,ALUNO DE PÓS-GRADUAÇÃO,1314272,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,747,254698,Belas artes,BIOQUÍMICA


In [7]:
#Extraindo e criando coluna de ano a partir da data de emprestimo
alunos_pg['ano'] = alunos_pg['data_emprestimo'].dt.year

In [8]:
#Criando um novo dataframe contendo a quantidade de emprestimos realizadas por ano em cada curso de graduação
total_curso_ano = alunos_pg.groupby(['ano','Curso'])['data_emprestimo'].value_counts().reset_index(name='qtds')
total_curso_ano

,ano,Curso,data_emprestimo,qtds
0,2017,ADMINISTRAÇÃO,2017-04-18 09:11:22.181,2
1,2017,ADMINISTRAÇÃO,2017-04-12 16:39:42.666,2
2,2017,ADMINISTRAÇÃO,2017-11-20 16:06:57.961,2
3,2017,ADMINISTRAÇÃO,2017-09-13 13:54:36.182,2
4,2017,ADMINISTRAÇÃO,2017-06-01 12:27:13.355,2
...,...,...,...,...
91824,2020,GESTÃO DA INFORMAÇÃO E DO CONHECIMENTO,2020-01-07 14:29:24.957,1
91825,2020,GESTÃO DA INFORMAÇÃO E DO CONHECIMENTO,2020-01-07 14:29:24.925,1
91826,2020,GESTÃO DA INFORMAÇÃO E DO CONHECIMENTO,2020-01-07 14:29:24.892,1
91827,2020,GESTÃO DA INFORMAÇÃO E DO CONHECIMENTO,2020-01-07 14:29:24.863,1


In [9]:
#Criando a tabela pivot formato 2
pivot_table = total_curso_ano.pivot_table(index='Curso', columns='ano', values='qtds', aggfunc='sum')
pivot_table

ano,2017,2018,2019,2020
Curso,,,,
ADMINISTRAÇÃO,2538,2316,1692,269
ARQUITETURA E URBANISMO,2209,1917,1818,257
ARTES CÊNICAS,2115,2082,1812,229
BIOINFORMÁTICA,2190,1914,1825,263
BIOQUÍMICA,2294,1944,2057,272
CIÊNCIA E ENGENHARIA DE MATERIAIS,2316,2110,2008,328
"CIÊNCIA, TECNOLOGIA E INOVAÇÃO",2415,2365,2026,303
CIÊNCIAS DA SAÚDE,2434,1986,1825,234
CIÊNCIAS ODONTOLÓGICAS,2414,2540,2134,352


In [10]:
#Importando informações com previsão de emprestimos para o ano de 2022 
previsao = pd.read_csv('/Users/Clarice Nascimento/0. Alura/d7/previsao.txt', delimiter = ' ')
previsao = previsao.rename(columns={'curso': 0, 'previsao_2022': 1})
previsao

,0,1
0,administracao,1905
1,arquitetura_e_urbanismo,1896
2,artes_cenicas,1920
3,bioinformatica,2302
4,bioquimica,2090
5,ciencia_e_engenharia_de_materiais,2010
6,ciencia_tecnologia_e_inovacao,1960
7,ciencias_da_saude,1730
8,ciencias_odontologicas,2405
9,design,1900


In [11]:
#Agregando a previsão de emprestimos de 2022 ao dataframe de observação
previsao.index = pivot_table.index
pivot_table['2022']  =  previsao.iloc[:,1]
pivot_table 

ano,2017,2018,2019,2020,2022
Curso,,,,,
ADMINISTRAÇÃO,2538,2316,1692,269,1905
ARQUITETURA E URBANISMO,2209,1917,1818,257,1896
ARTES CÊNICAS,2115,2082,1812,229,1920
BIOINFORMÁTICA,2190,1914,1825,263,2302
BIOQUÍMICA,2294,1944,2057,272,2090
CIÊNCIA E ENGENHARIA DE MATERIAIS,2316,2110,2008,328,2010
"CIÊNCIA, TECNOLOGIA E INOVAÇÃO",2415,2365,2026,303,1960
CIÊNCIAS DA SAÚDE,2434,1986,1825,234,1730
CIÊNCIAS ODONTOLÓGICAS,2414,2540,2134,352,2405


In [12]:
#Função para calculo percentual de variação
def diferenca_percentual_ano_anterior(x,y):
  return round(((x / y) - 1)*100,2)

In [13]:
#Calculo das diferenças percentuais
df_2018 = diferenca_percentual_ano_anterior(pivot_table.iloc[:,1], pivot_table.iloc[:,0])
df_2019 = diferenca_percentual_ano_anterior(pivot_table.iloc[:,2], pivot_table.iloc[:,1])
df_2022 = diferenca_percentual_ano_anterior(pivot_table.iloc[:,3], pivot_table.iloc[:,2])

In [14]:
#Alocando os calculos em um dataframe
percentual = pd.DataFrame({'2018':df_2018,
                           '2019':df_2019,
                           '2022':df_2022})
percentual

,2018,2019,2022
Curso,,,
ADMINISTRAÇÃO,-8.75,-26.94,-84.10
ARQUITETURA E URBANISMO,-13.22,-5.16,-85.86
ARTES CÊNICAS,-1.56,-12.97,-87.36
BIOINFORMÁTICA,-12.60,-4.65,-85.59
BIOQUÍMICA,-15.26,5.81,-86.78
CIÊNCIA E ENGENHARIA DE MATERIAIS,-8.89,-4.83,-83.67
"CIÊNCIA, TECNOLOGIA E INOVAÇÃO",-2.07,-14.33,-85.04
CIÊNCIAS DA SAÚDE,-18.41,-8.11,-87.18
CIÊNCIAS ODONTOLÓGICAS,5.22,-15.98,-83.51


In [15]:
#Formatando tabela final
percentual.reset_index(inplace=True)
percentual.columns = percentual.columns.str.capitalize()
percentual.Curso = percentual.Curso.str.capitalize()
percentual

,Curso,2018,2019,2022
0,Administração,-8.75,-26.94,-84.10
1,Arquitetura e urbanismo,-13.22,-5.16,-85.86
2,Artes cênicas,-1.56,-12.97,-87.36
3,Bioinformática,-12.60,-4.65,-85.59
4,Bioquímica,-15.26,5.81,-86.78
5,Ciência e engenharia de materiais,-8.89,-4.83,-83.67
6,"Ciência, tecnologia e inovação",-2.07,-14.33,-85.04
7,Ciências da saúde,-18.41,-8.11,-87.18
8,Ciências odontológicas,5.22,-15.98,-83.51
9,Design,-19.10,0.16,-86.20
